In [1]:
import torch
from torch import nn
import math
import numpy as np

from torch.autograd import Variable
import torch.functional as F
import torch.nn.functional as F
from torch.utils.data import DataLoader
torch.manual_seed(0)
import re
import pandas as pd

import copy

Cloning into 'stepik-dl-nlp'...
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 273 (delta 3), reused 5 (delta 2), pack-reused 266
Receiving objects: 100% (273/273), 42.13 MiB | 21.35 MiB/s, done.
Resolving deltas: 100% (132/132), done.
     |████████████████████████████████| 51kB 2.5MB/s 
     |████████████████████████████████| 1.7MB 7.1MB/s 
     |████████████████████████████████| 307kB 33.8MB/s 
     |████████████████████████████████| 7.1MB 28.8MB/s 
  Created wheel for ufal.udpipe: filename=ufal.udpipe-1.2.0.3-cp36-cp36m-linux_x86_64.whl size=5625263 sha256=baeac858711df035423d76e1fae5e009d5c5901666ecc38051bfd6f755b5b411
  Stored in directory: /root/.cache/pip/wheels/0c/9d/db/6d3404c33da5b7adb6c6972853efb6a27649d3ba15f7e9bebb
  Created wheel for intervaltree: filename=intervaltree-3.0.2-cp36-none-any.whl size=25791 sha256=6a242ce93ab6bdefe73b535be9808e508f72f8efe4495ee35f2e884deffeb65b
  

In [77]:
def tokenize_text_simple_regex(txt, min_token_size=4):
    TOKEN_RE = re.compile(r'[\w\d]+')
    txt = txt.lower()
    all_tokens = TOKEN_RE.findall(txt)
    return [token for token in all_tokens if len(token) >= min_token_size]

def tokenize_corpus(texts, tokenizer=tokenize_text_simple_regex, **tokenizer_kwargs):
    return [tokenizer(text, **tokenizer_kwargs) for text in texts]

def split_into_chunks(tokens, size):
  result = list()
  j = 0
  for i in range(1,len(tokens)):
    if (i%size == 0):
      result.append(tokens[j:i])
      j = i
  result.append(tokens[j:i+1])
  return result

dataset = "A language is a structured system of communication. Language, in a broader sense, is the method of communication that involves the use of – particularly human – languages. The scientific study of language is called linguistics. Questions concerning the philosophy of language, such as whether words can represent experience, have been debated at least since Gorgias and Plato in ancient Greece. Thinkers such as Rousseau have argued that language originated from emotions while others like Kant have held that it originated from rational and logical thought. Twentieth century philosophers such as Wittgenstein argued that philosophy is really the study of language. Major figures in linguistics include Ferdinand de Saussure and Noam Chomsky."
dataset = split_into_chunks(tokenize_corpus([dataset])[0], 10)
print(dataset)

[['language', 'structured', 'system', 'communication', 'language', 'broader', 'sense', 'method', 'communication', 'that'], ['involves', 'particularly', 'human', 'languages', 'scientific', 'study', 'language', 'called', 'linguistics', 'questions'], ['concerning', 'philosophy', 'language', 'such', 'whether', 'words', 'represent', 'experience', 'have', 'been'], ['debated', 'least', 'since', 'gorgias', 'plato', 'ancient', 'greece', 'thinkers', 'such', 'rousseau'], ['have', 'argued', 'that', 'language', 'originated', 'from', 'emotions', 'while', 'others', 'like'], ['kant', 'have', 'held', 'that', 'originated', 'from', 'rational', 'logical', 'thought', 'twentieth'], ['century', 'philosophers', 'such', 'wittgenstein', 'argued', 'that', 'philosophy', 'really', 'study', 'language'], ['major', 'figures', 'linguistics', 'include', 'ferdinand', 'saussure', 'noam', 'chomsky']]


In [78]:
dataset[0]

['language',
 'structured',
 'system',
 'communication',
 'language',
 'broader',
 'sense',
 'method',
 'communication',
 'that']

In [79]:
def make_word2idx(D):
  result = dict()
  result['<PAD>'] =0
  i = 1
  for d in D:
    for w in d:
      if w not in result:
        result[w] = i
        i += 1
  return result

def encode(D, word2idx, dim):
  result = torch.zeros(size=(len(D), dim))
  print(result.shape)
  for i in range(len(D)):
    for j in range(len(D[i])):
      result[i][j] = word2idx[D[i][j]]
  return result


word2idx = make_word2idx(dataset)
train_data = encode(dataset, word2idx, 10)



torch.Size([8, 10])


In [80]:
word2idx

{'<PAD>': 0,
 'ancient': 32,
 'argued': 36,
 'been': 26,
 'broader': 5,
 'called': 15,
 'century': 49,
 'chomsky': 59,
 'communication': 4,
 'concerning': 18,
 'debated': 27,
 'emotions': 39,
 'experience': 24,
 'ferdinand': 56,
 'figures': 54,
 'from': 38,
 'gorgias': 30,
 'greece': 33,
 'have': 25,
 'held': 44,
 'human': 11,
 'include': 55,
 'involves': 9,
 'kant': 43,
 'language': 1,
 'languages': 12,
 'least': 28,
 'like': 42,
 'linguistics': 16,
 'logical': 46,
 'major': 53,
 'method': 7,
 'noam': 58,
 'originated': 37,
 'others': 41,
 'particularly': 10,
 'philosophers': 50,
 'philosophy': 19,
 'plato': 31,
 'questions': 17,
 'rational': 45,
 'really': 52,
 'represent': 23,
 'rousseau': 35,
 'saussure': 57,
 'scientific': 13,
 'sense': 6,
 'since': 29,
 'structured': 2,
 'study': 14,
 'such': 20,
 'system': 3,
 'that': 8,
 'thinkers': 34,
 'thought': 47,
 'twentieth': 48,
 'whether': 21,
 'while': 40,
 'wittgenstein': 51,
 'words': 22}


## Skip-Gram architecture

In the Skip-Gram model, the goal is to predict a context with the given word. For each word we have two vectors: 

$w_1 \in W_1 \in R^{Vocab \times EmbDim} $ it defines the word itself

$w_2 \in W_2$ it defines the word's context.

Defines objective function to maximize:

$max \, \Pi_{center} \Pi_{context} P(context|center, \theta) $

This is not suitable to compute, so we replace probability with negative log likelihood. We can do it for $log$ is monotonic function and extrema does not change its position.

Get

$\min_{\theta} - log \Pi_{center} \Pi_{context} P(context|center, \theta)$

Let us replace products with sums using log-proprities and devide it by number of pairs $T$

$L = -\frac{1}{T} \sum_{center} \sum_{context} log P(context|center, \theta)$

Let us now define probability function $P$. Let us assume that each word is represented as two vectors: $v$ - vector of word itself and $u$ - vector of its context. We will use SoftMax :

$P = \frac{exp(w_1^T w_2)}{\sum_{w_1 \in vocab} exp(w_1^T w_2)}$

Thus, for each existing center, context pair in corpus we’re computing their “similarity score”. And divide it by sum of each theoretically possible context — to know whether score is relatively high or low. As softmax is guaranteed to take a value between 0 and 1 it defines a valid probability distribution.







## Training

Train data consists of pairs [$w_t$, [ $w_{c_1}$ , $w_{c_2}$ , ... ]], where
$w_t$ is a target word and $w_{c_i}$ is context for the target word.

The Word2Vec model consists of 2 weight matrices $w_1 \in M_{n,m}$ and $w_2 \in M_{m,n}$ where $n$ is size of vocabulary and $m$ is size of train data

*Error* is defined to be a distance between predicted context $y$ and $w_c$

*Backpropagation* is a function for adjustment we need to alter the weights.

$\frac{\partial g(x)}{\partial w_i} = \frac{\partial f^1(W^1(f^2(W^2))}{\partial w_i} $ 


In [81]:
# make symmetric mask matrix M, where all elements are zeroes but M[i][1] = 1, ... , M[i][r] = 1, 
# where r - radius, i in range(0, N)  
def make_diag_mask(N, radius):
  idx = torch.arange(N)
  abs_idx_diff = (idx.unsqueeze(0) - idx.unsqueeze(1)).abs()
  mask = ((abs_idx_diff <= radius ) & (abs_idx_diff > 0)).float() # fill it with True or False
  return mask

# this mask allows us to define a context's window


In [82]:
class Word2Vec(nn.Module):
  def __init__(self, vocab_size, emb_size, sent_len, radius=5, negative_samples_n=5):
    super().__init__()
    self.vocab_size = vocab_size
    self.negative_samples_n = negative_samples_n

    self.W1 = nn.Embedding(self.vocab_size, emb_size, padding_idx=0)
    self.W1.weight.data.uniform_(-1.0 / emb_size, 1.0 / emb_size ) # by default there is a normal noise, so we change it
    self.W1.weight.data[0] = 0 # ped

    self.W2 = nn.Embedding(self.vocab_size, emb_size, padding_idx=0)
    self.W2.weight.data.uniform_(-1.0 / emb_size, 1.0 / emb_size )
    self.W2.weight.data[0] = 0

    self.mask = make_diag_mask(sent_len, radius)


  def forward(self, sents):
    '''
    sents : int tensor [Batch x MaxSentLen] - ids of words
    '''

    batch_size = sents.shape[0]
    center = self.W1(sents) # by a batch of encoded sentences it returns emeddings for words of these sentences [Batch x MaxSentLen x EmbSize] 

    # since we want to build a classifier that tells us whether words can occure in one context or not, we need to make positive and negative examples

    # positive:
    # evaluate similarity with true context words.
    pos_context_embs = self.W2(sents).permute(0,2,1) # transpose it in order to make dot product with center embeddings
    pos_sims = torch.bmm(center, pos_context_embs) # score of similarity for each pairs of words, for example, positive_sims[i][j] - vector of similarity score of jth word in ith document
    pos_probs = torch.sigmoid(pos_sims) # convert similarity scores into probabilities over pairs of words; sigmoid function is applied elementwise
    
    # but we want to count only these scores that are in certain window; thus, we apply mask matrix
    self.pos_mask = self.mask.to('cpu')


    # negative:
    # take random words
    neg_words = torch.randint(1,self.vocab_size,size =(batch_size, self.negative_samples_n), device='cpu') # Batch x NegSamplesN
    neg_context_embs = self.W2(neg_words).permute(0,2,1)
    neg_sims = torch.bmm(center, neg_context_embs)
    neg_probs = torch.sigmoid(neg_sims)

    return pos_probs, neg_probs
  


In [83]:
def copy_data_to_device(data, device):
    if torch.is_tensor(data):
        return data.to(device)
    elif isinstance(data, (list, tuple)):
        return [copy_data_to_device(elem, device) for elem in data]


def train(model, train_dataset,
                    lr=1e-4, epoch_n=10, batch_size=32,
                    device=None, early_stopping_patience=10, l2_reg_alpha=0,
                    max_batches_per_epoch_train=10000,
                    lr_scheduler_ctor=None,
                    shuffle_train=True,
                    dataloader_workers_n=0):
    """ 
    params
    ---

    model: torch.nn.Module - model to train
    train_dataset: torch.utils.data.Dataset
    data_loader_ctor: функция для создания объекта, преобразующего датасет в батчи
        (по умолчанию torch.utils.data.DataLoader)

    returns
    ---
    best_model : torch.nn.Module

    """

    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=l2_reg_alpha)
    
    train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=shuffle_train,
                                        num_workers=dataloader_workers_n)
    

    best_epoch_i = 0
    best_model = copy.deepcopy(model)

    for epoch_i in range(epoch_n):
            print('Epoch {}'.format(epoch_i))

            model.train()
            mean_train_loss = 0
            train_batches_n = 0
            for batch_i, batch_x in enumerate(train_dataloader):
                if batch_i > max_batches_per_epoch_train:
                    break

                batch_x = copy_data_to_device(batch_x, device)

                pos_prob, neg_prob = model.forward(batch_x)
                
                # since we are solving classification problem, we use binary cross entropy and try to MAXIMIZE it
                pos_loss = F.binary_cross_entropy(pos_prob * model.pos_mask, model.pos_mask.expand_as(pos_prob)) # second argument is target

                # in this case, we try to MINIMIZE it.
                neg_loss = F.binary_cross_entropy(neg_prob, neg_prob.new_zeros(neg_prob.shape)) # zeros as targets

                loss = pos_loss + neg_loss

                model.zero_grad()
                loss.backward()

                optimizer.step()

                mean_train_loss += float(loss)
                train_batches_n += 1

            mean_train_loss /= train_batches_n
            print('Epoch: {} Iterations'.format(train_batches_n))
            print('LOSS = ', mean_train_loss)


            model.eval()
            mean_val_loss = 0
            val_batches_n = 0
            
    return model

In [92]:
model = Word2Vec(vocab_size=len(word2idx), emb_size=25, sent_len=10, radius=4)
new_model = train(model, train_data.long(), lr=0.1, epoch_n = 20, batch_size = 2, device='cpu')

Epoch 0
Epoch: 4 Iterations
LOSS =  1.111543208360672
Epoch 1
Epoch: 4 Iterations
LOSS =  1.1333697140216827
Epoch 2
Epoch: 4 Iterations
LOSS =  0.9851695001125336
Epoch 3
Epoch: 4 Iterations
LOSS =  1.0932619273662567
Epoch 4
Epoch: 4 Iterations
LOSS =  1.1448953747749329
Epoch 5
Epoch: 4 Iterations
LOSS =  1.03008933365345
Epoch 6
Epoch: 4 Iterations
LOSS =  1.0044880211353302
Epoch 7
Epoch: 4 Iterations
LOSS =  0.9577254205942154
Epoch 8
Epoch: 4 Iterations
LOSS =  1.2059921324253082
Epoch 9
Epoch: 4 Iterations
LOSS =  1.1750897467136383
Epoch 10
Epoch: 4 Iterations
LOSS =  1.0724432468414307
Epoch 11
Epoch: 4 Iterations
LOSS =  0.9453153610229492
Epoch 12
Epoch: 4 Iterations
LOSS =  1.231875628232956
Epoch 13
Epoch: 4 Iterations
LOSS =  0.8457540571689606
Epoch 14
Epoch: 4 Iterations
LOSS =  0.8232067376375198
Epoch 15
Epoch: 4 Iterations
LOSS =  0.7896745949983597
Epoch 16
Epoch: 4 Iterations
LOSS =  0.7182120680809021
Epoch 17
Epoch: 4 Iterations
LOSS =  0.987585574388504
Epoch 1

In [70]:
def similarity(v,u):
  return torch.dot(v,u)/(torch.norm(v)*torch.norm(u))

In [96]:
similarity(new_model.W1.weight[word2idx["chomsky"]], new_model.W1.weight[word2idx["saussure"]])

tensor(0.5768, grad_fn=<DivBackward0>)